In [37]:
import tensorflow as tf

import numpy as np
import os
import time

In [38]:
text = open('poem.txt',"r", errors='ignore').read()
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 16448 characters


In [39]:
print(text[:250])

All alone in the mid of night,
I think what I could strike,
I remember the day I saw her,
Walking down all draped in her furr,
She looked quizzed, I was puzzled,
That Bell rang with no sense whatsoever,
I tried to hold my heart,
But it slipped like h


In [40]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

69 unique characters


In [41]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [42]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '/' :  10,
  '2' :  11,
  ';' :  12,
  '?' :  13,
  'A' :  14,
  'B' :  15,
  'C' :  16,
  'D' :  17,
  'E' :  18,
  'F' :  19,
  ...
}


In [43]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

A
l
l
 
a


In [44]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'All alone in the mid of night,\nI think what I could strike,\nI remember the day I saw her,\nWalking dow'
'n all draped in her furr,\nShe looked quizzed, I was puzzled,\nThat Bell rang with no sense whatsoever,'
'\nI tried to hold my heart,\nBut it slipped like hands on a slippery wall,\nI thought I could make her s'
'mile,\nTo my surprise, she laughed and I revived,\nGame of shadow turned my life,\nShe grabbed my hand o'
'n a Thursday Night,\nI so thought that it could be "the night",\nBut I read her wrong and missed my lin'


In [45]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [46]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'All alone in the mid of night,\nI think what I could strike,\nI remember the day I saw her,\nWalking do'
Target data: 'll alone in the mid of night,\nI think what I could strike,\nI remember the day I saw her,\nWalking dow'


In [47]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 14 ('A')
  expected output: 48 ('l')
Step    1
  input: 48 ('l')
  expected output: 48 ('l')
Step    2
  input: 48 ('l')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 37 ('a')
Step    4
  input: 37 ('a')
  expected output: 48 ('l')


In [48]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [50]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [51]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [52]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [53]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 69) # (batch_size, sequence_length, vocab_size)


In [54]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           17664     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 69)            70725     
Total params: 4,026,693
Trainable params: 4,026,693
Non-trainable params: 0
_________________________________________________________________


In [55]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [56]:
sampled_indices

array([31, 60, 16, 62, 15, 62, 54, 45, 16, 23, 29, 16, 21, 27, 47,  1, 11,
       52, 67,  4, 22, 20, 45, 39,  3, 36,  0, 48, 21, 17, 21,  1, 16, 61,
       36, 31, 53, 33, 36, 41, 11, 38, 39, 28, 64, 61, 41,  6, 12, 48, 11,
       46, 64, 61, 68, 50,  9, 46, 53, 56, 38, 43, 13, 15,  3, 57, 67, 21,
       15, 10, 19, 40, 40, 55, 24, 34, 30, 10, 24, 19, 63, 14, 30, 23, 40,
       23, 51,  6, 53, 65, 42, 46, 44, 63, 57, 16,  5, 62, 58, 68],
      dtype=int64)

In [57]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ze if you love someone when they are near you, maybe not in every single case, but when they leave y'

Next Char Predictions: 
 'SxCzBzriCJPCHNk 2p€\'IGic"Z\nlHDH CyZSqVZe2bcOœye);l2jœy™n.jqtbg?B"u€HB/FddsKWR/KFâARJdJo)q˜fjhâuC(zv™'


In [58]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 69)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.233695


In [59]:
model.compile(optimizer='adam', loss=loss)

In [60]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [61]:
EPOCHS=10

In [62]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
2/2 [==============================] - 5s 3s/step - loss: 4.2111
Epoch 2/10
2/2 [==============================] - 5s 3s/step - loss: 4.0454
Epoch 3/10
2/2 [==============================] - 5s 3s/step - loss: 7.0614
Epoch 4/10
2/2 [==============================] - 6s 3s/step - loss: 3.6879
Epoch 5/10
2/2 [==============================] - 6s 3s/step - loss: 3.7174
Epoch 6/10
2/2 [==============================] - 6s 3s/step - loss: 3.6896
Epoch 7/10
2/2 [==============================] - 6s 3s/step - loss: 3.5534
Epoch 8/10
2/2 [==============================] - 6s 3s/step - loss: 3.4856
Epoch 9/10
2/2 [==============================] - 6s 3s/step - loss: 3.3500
Epoch 10/10
2/2 [==============================] - 6s 3s/step - loss: 3.1989


In [63]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [64]:
'./training_checkpoints/ckpt_10'

'./training_checkpoints/ckpt_10'

In [65]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [66]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            17664     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 69)             70725     
Total params: 4,026,693
Trainable params: 4,026,693
Non-trainable params: 0
_________________________________________________________________


In [67]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"Love"))

LovetKteI nt™cegfeoie atoiaa nunedsdttle wdtgs sncoenlaooa naatprsSurrtsiooasebctoAepuuasovDers,rin esrieovomr  ruieoparso™mhakoeu.h eihmousahaqmte ,itibarqeoueht,tthtofnah
vornrnisiame,bumeeelt dhhan iueetamys-yocohptoytc,rmselVcer tr tticoiooweotra,epsnnto;nvit md
aneunFiheatKlabrsen bthsy
 Onenoimtii sttidtssiy" giyr d hwite inieiilbho?y ssremarinlte astnpi
i ayn ?E i  teoen aigyarrwllirleroeninhsldwis aendeeoiuonYgnstneiu,eioegw.dpiAnvtmyrub binrnarnynotrnnenihnoonraencot uoh ul €tmmstientoyiav cornmclicanhmanmia ihes antooisnlnor uoladsosa inannLtsieyloiia,"dhouisyornI smotprcsooaeyisesgleveoetioseyoevh snm amlaesuoeheyarwans.rmIlidnhdmo?hraye ahtnstaI kdpnœarstinbaaiin estessAnnernaiatrhchsda Cesthfes.iyaytlycort€ovef hnicndl,snitaaae
Idti˜Ase iuaiah€TornlwdY eheeaydintoVrn hgmttcoleasrntate rlelwianesdoNimiutuhgereaousenhrhtnswgststdettimZvtiyoeeystrnmiaodnihidiolop,ianrhale dcesesnm eitlbpeioosl aothdsoihdehaudoteeneiaiamo anhoiaehiotdbnauvlyeorldseercesu helidaaelsrederpaansSy